## Recurrent Neural Network (RNN) - Deep Learning
Son redes de aprendizaje profundo cuyo propósito se enmarca en la predicción basada en series de tiempo, son particularmente usadas en procesamiento de video y en tareas de NLP para sugerir palabras (text completion). No obstante pueden usarse para generación de expresiones matemáticas con un enfoque relativamente nuevo propuesto aquí: https://openreview.net/pdf?id=m5Qsh0kBQG

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.utils.random import check_random_state
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler



### Ejemplo RNN
Se aplicará una RNN para predecir la distancia entre un par de puntos (x,y) y (x1,y1), uno de ellos constante.$$ d = \sqrt{(x1-x)^2 + (y1 - y)^2} $$ <br>
Como son redes que aplican para problemas basados en series de tiempo, se decidió que cada 100 registros se haría un timestep para especificar el número de observaciones previas cuando la RNN haga una predicción sobre la observación actual.<br>
### Generar el conjunto de datos
Para generar el conjunto de datos se calculará la distancia entre un conjunto de 5000 puntos (x,y) aleatorios, así, el modelo de regresión se ajustará a una proporción del 80% del conjunto y el 20% restante estará destinado a la predicción.

In [6]:
def calcular_distancia(X,Y,XF,YF):
    return np.round(np.sqrt(np.power(XF-X,2)+np.power(YF-Y,2)),2)

XF = 5
YF = 12.5

rng = check_random_state(0)

X = np.round(rng.uniform(1, 11, 10000).reshape(5000, 2),2)
XFs=[[XF] for i in range(5000)]
YFs=[[YF] for i in range(5000)]
X=np.append(X, XFs, axis=1)
X=np.append(X, YFs, axis=1)
Y = calcular_distancia(X[:, 0],X[:, 1],XF,YF)
Y = Y.reshape(-1,1)

#training_data = np.append(X,Y, axis=1)
training_data = Y

scaler = MinMaxScaler()

#training_data = scaler.fit_transform(training_data)
training_data = scaler.fit_transform(training_data.reshape(-1, 1))#Estirarlo

##TIMESTEPS
x_training_data = []
y_training_data =[]
for i in range(100, len(training_data)):
    x_training_data.append(training_data[i-100:i, 0])
    y_training_data.append(training_data[i, 0])

x_training_data = np.array(x_training_data)
y_training_data = np.array(y_training_data)

print(x_training_data.shape)
print(y_training_data.shape)

x_training_data = np.reshape(x_training_data, (x_training_data.shape[0], x_training_data.shape[1], 1))
print(x_training_data.shape)

rnn = Sequential()
rnn.add(LSTM(units = 45, return_sequences = True, input_shape = (x_training_data.shape[1], 1)))
rnn.add(Dropout(0.2))
for i in [True, True, False]:
    rnn.add(LSTM(units = 45, return_sequences = i))
    rnn.add(Dropout(0.2))
    
rnn.add(Dense(units = 1))
rnn.compile(optimizer = 'adam', loss = 'mean_squared_error', verbose=0)
rnn.fit(x_training_data, y_training_data, epochs = 100, batch_size = 32)


(4900, 100)
(4900,)
(4900, 100, 1)


TypeError: Invalid keyword argument(s) in `compile`: {'verbose'}

### Proceso de transformación
Es una práctica común, en las redes neuronales, normalizar el conjunto de datos de entrenamiento, pues se ha demostrado que las redes neuronales aprenden mejor con valores pequeños entre 0 y 1.

In [15]:
def transformar(X,Y):
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    x_scale = scaler_x.fit_transform(X)
    y_scale = scaler_y.fit_transform(Y)
    return (x_scale,y_scale)